In [1]:
# TODO address all TODOs in dovs_and_summary_usage_helper_functions.ipynb
%run dovs_and_summary_usage_helper_functions.ipynb

In [2]:
from importlib import reload
#reload(Utilities)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import itertools
import copy
import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
from EEMSP import EEMSP
#-----
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_Box_sns
import GrubbsTest

# -----------------------------------------------------------------------------------------------
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# -----------------------------------------------------------------------------------------------

In [3]:
# TODO Get rid of inactive meters

In [4]:
save_dfs_to_csv = False
read_dfs_from_csv = True
save_dir_csvs = os.path.join(Utilities.get_local_data_dir(), r'dovs_and_summary_usage_data')

save_figs = False
#save_dir_base_figs = r'C:\Users\s346557\Documents\Presentations\GroupMeetings\20220120\Figures'
save_dir_base_figs = r'C:\Users\s346557\Documents\Presentations\OutagePredictions\MeetingWithEngineersEtc_20220127\Figures'

assert(save_dfs_to_csv+read_dfs_from_csv <=1) # Should never both read and write!

In [5]:
fig_num = 0

In [6]:
if not read_dfs_from_csv:
    conn_outages = Utilities.get_utldb01p_oracle_connection()
    conn_aws = Utilities.get_athena_prod_aws_connection()

In [7]:
date_0_no_outg = '2018-01-01'
date_1_no_outg = '2021-12-31'

# date_0_no_outg = '2019-01-01'
# date_1_no_outg = '2019-06-30'

In [8]:
date_0_outg = '2018-01-01'
date_1_outg = '2021-12-31'

# date_0_outg = '2019-01-01'
# date_1_outg = '2019-06-30'

# -----------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------

# Build no outage collection and df_mp_all
where df_mp_all is a DataFrame containing all meter premise data (from OH)

In [9]:
# cols_of_interest_met_prem = [
#     'mfr_devc_ser_nbr',
#     'longitude',
#     'latitude',
#     'state_cd', 
#     'prem_nb',
#     'srvc_pole_nb',
#     'trsf_pole_nb',
#     'latitude_nb',
#     'longitude_nb',
#     'station_nb', 
#     'xfmr_nb', 
#     'annual_kwh',
#     'annual_max_dmnd'
# ]

# cols_of_interest_met_prem = [
#     'mfr_devc_ser_nbr',
#     'state_cd', 
#     'prem_nb',
#     'trsf_pole_nb',
#     'annual_kwh',
#     'annual_max_dmnd'
# ]

cols_of_interest_met_prem = [
    'mfr_devc_ser_nbr',
    'state_cd', 
    'prem_nb',
    'trsf_pole_nb',
    'annual_kwh',
    'annual_max_dmnd', 
    'mtr_stat_cd',
    'mtr_stat_cd_desc', 
    'devc_stat_cd', 
    'devc_stat_cd_desc'
]

# Find transformers which haven't experienced an outage since date_0_no_outg

### First, find all transformers which HAVE experienced an outage

In [10]:
sql_outage_location_ids = DOVSOutages_SQL.build_sql_find_outage_xfmrs(
    mjr_mnr_cause=None, 
    include_premise=False, 
    date_range=[date_0_no_outg, date_1_no_outg], 
    states='OH'
)
sql_stmnt_outage_location_ids = sql_outage_location_ids.get_sql_statement()
print(sql_stmnt_outage_location_ids)

SELECT
	DISTINCT(DOV.LOCATION_ID)
FROM DOVSADM.DOVS_OUTAGE_FACT DOV
	LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
	LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
	LEFT OUTER JOIN DOVSADM.DOVS_CLEARING_DEVICE_DIM DOV3 ON DOV.DEVICE_CD=DOV3.DEVICE_CD
	LEFT OUTER JOIN DOVSADM.DOVS_EQUIPMENT_TYPES_DIM DOV4 ON DOV.EQUIPMENT_CD=DOV4.EQUIPMENT_CD
	LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_CAUSE_TYPES_DIM DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
WHERE DOV.DT_OFF_TS BETWEEN '2018-01-01' AND '2021-12-31'
AND   DOV2.INTRPTN_TYP_CD = 'S'
AND   DOV2.CURR_REC_STAT_CD = 'A'
AND   DOV.MJR_CAUSE_CD <> 'NI'
AND   DOV.DEVICE_CD <> 85
AND   DOV.STATE_ABBR_TX = 'OH'


## Find all premises which have suffered outage?

In [11]:
find_outage_premises=False
if find_outage_premises:
    sql_outage_premises = DOVSOutages_SQL.build_sql_distinct_outage_field(
        distinct_field=SQLSelectElement(field_desc='PREMISE_NB', 
                                        table_alias_prefix='PRIM'), 
        mjr_mnr_cause=None, 
        include_premise=True, 
        date_range=[date_0_no_outg, date_1_no_outg], 
        states='OH'
    )
    sql_stmnt_outage_premises = sql_outage_premises.get_sql_statement()
    sql_outage_premises.print_sql_statement()
    
    df_outage_premises = pd.read_sql_query(sql_stmnt_outage_premises, conn_outages)
    df_outage_premises.head()

In [12]:
compare_to_og = False
if compare_to_og:
    if 'conn_outages' not in locals():
        conn_outages = Utilities.get_utldb01p_oracle_connection()  
    sql_outage_location_ids_0 = (
    """
    SELECT   DISTINCT(DOV.LOCATION_ID)
    FROM     DOVSADM.DOVS_OUTAGE_FACT DOV
             LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
             LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
             LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_CAUSE_TYPES_DIM DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
    WHERE    DOV.DT_OFF_TS BETWEEN {} AND '{}'
    AND  DOV2.INTRPTN_TYP_CD = 'S'
    AND  DOV2.CURR_REC_STAT_CD = 'A' 
    AND  DOV.MJR_CAUSE_CD <> 'NI' 
    AND  DOV.DEVICE_CD <> 85 
    AND  DOV.STATE_ABBR_TX = 'OH'
    """
    ).format(f"'{date_0_no_outg}'", date_1_no_outg)

    print(sql_outage_location_ids_0)

    df_outage_location_ids_0 = pd.read_sql_query(sql_outage_location_ids_0, conn_outages)
    df_outage_location_ids = pd.read_sql_query(sql_stmnt_outage_location_ids, conn_outages)
    print(df_outage_location_ids.sort_values(by='LOCATION_ID', ignore_index=True).equals(df_outage_location_ids_0.sort_values(by='LOCATION_ID', ignore_index=True)))

In [13]:
if read_dfs_from_csv:
    df_outage_location_ids_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_outage_location_ids_OG.csv'), dtype=str)
else:
    df_outage_location_ids_OG = pd.read_sql_query(sql_stmnt_outage_location_ids, conn_outages)
print(df_outage_location_ids_OG.shape)
df_outage_location_ids = df_outage_location_ids_OG.copy()

(107899, 1)


### Now, find those which HAVEN'T experienced an outage

### When df_outage_location_ids is large
i.e., when the date range used to find outages is large
<br> It is actually faster to grab ALL meters from default.meter_premise and then filter using pandas
<br> as opposed to using NOT IN and filtering with SQL

In [14]:
sql_mp_all = MeterPremise.build_sql_meter_premise(cols_of_interest_met_prem, states=['OH']).get_sql_statement()
print(sql_mp_all)

SELECT
	mfr_devc_ser_nbr,
	state_cd,
	prem_nb,
	trsf_pole_nb,
	annual_kwh,
	annual_max_dmnd,
	mtr_stat_cd,
	mtr_stat_cd_desc,
	devc_stat_cd,
	devc_stat_cd_desc
FROM default.meter_premise


In [15]:
compare_to_og = False
if compare_to_og:
    if 'conn_aws' not in locals():
        conn_aws = Utilities.get_athena_prod_aws_connection()
    sql_mp_all_0 = (
    """
    SELECT {}
    FROM default.meter_premise
    WHERE state_cd IN ('OH')
    """
    ).format(','.join(cols_of_interest_met_prem))
    print(sql_mp_all_0)

    df_mp_all_0 = pd.read_sql(sql_mp_all_0, conn_aws)
    df_mp_all = pd.read_sql(sql_mp_all, conn_aws)
    print(df_mp_all.sort_values(by=['mfr_devc_ser_nbr'], ignore_index=True).equals(df_mp_all_0.sort_values(by=['mfr_devc_ser_nbr'], ignore_index=True)))

In [16]:
if read_dfs_from_csv:
    df_mp_all_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_mp_all_OG.csv'), dtype=str)
else:
    df_mp_all_OG = pd.read_sql(sql_mp_all, conn_aws) 
df_mp_all_OG = Utilities_df.remove_prepend_from_columns_in_df(df_mp_all_OG)
print(df_mp_all_OG.shape)
df_mp_all = df_mp_all_OG.copy()

(1536914, 10)


In [17]:
# cols_of_interest_met_prem_2 = [
#     'mfr_devc_ser_nbr',
#     'state_cd', 
#     'prem_nb',
#     'trsf_pole_nb',
#     'annual_kwh',
#     'annual_max_dmnd', 
#     'mtr_stat_cd',
#     'mtr_stat_cd_desc', 
#     'devc_stat_cd', 
#     'devc_stat_cd_desc'
# ]

# sql_mp_all_2 = (
# """
# SELECT {}
# FROM default.meter_premise
# WHERE state_cd IN ('OH')
# """
# ).format(','.join(cols_of_interest_met_prem_2))
# #print(sql_mp_all_2)
# df_mp_all_2_OG = pd.read_sql(sql_mp_all_2, conn_aws) 
# df_mp_all_2_OG = Utilities_df.remove_prepend_from_columns_in_df(df_mp_all_2_OG)
# print(df_mp_all_2_OG.shape)
# df_mp_all_2 = df_mp_all_2_OG.copy()

In [18]:
df_mp_no_outg_OG = df_mp_all[~df_mp_all['trsf_pole_nb'].isin(df_outage_location_ids['LOCATION_ID'].tolist())]
print(df_mp_no_outg_OG.shape)
df_mp_no_outg = df_mp_no_outg_OG.copy()

(1172938, 10)


In [19]:
df_mp_no_outg

,mfr_devc_ser_nbr,state_cd,prem_nb,trsf_pole_nb,annual_kwh,annual_max_dmnd,mtr_stat_cd,mtr_stat_cd_desc,devc_stat_cd,devc_stat_cd_desc
0,99999100,OH,071682817,TRANSMISSION,6102000.000,1472.000,C,CONNECTED,A,ACTIVE
1,11847204,OH,079403597,41820782D20123,13495.000,0.000,C,CONNECTED,A,ACTIVE
2,14308968,OH,079195860,41821097C30137,24581.000,0.000,C,CONNECTED,A,ACTIVE
3,15325985,OH,070473618,41840848000016,4207.000,0.000,C,CONNECTED,A,ACTIVE
4,15995460,OH,077655454,41810925000130,7863.000,0.000,C,CONNECTED,A,ACTIVE
...,...,...,...,...,...,...,...,...,...,...
1536909,LW1312A593,OH,103071757,TRANSMISSION,NaN,NaN,C,CONNECTED,U,COMPANY USE DEVICE
1536910,LW1312A594,OH,103071757,TRANSMISSION,NaN,NaN,C,CONNECTED,U,COMPANY USE DEVICE
1536911,LW1312A595,OH,103071757,TRANSMISSION,NaN,NaN,C,CONNECTED,U,COMPANY USE DEVICE
1536912,LW1312A596,OH,103071757,TRANSMISSION,NaN,NaN,C,CONNECTED,U,COMPANY USE DEVICE


In [20]:
# Below, TC stands for Time Consuming
# Probably don't run unless date_0_no_outg and date_1_no_outg differ by no more than
# a day or two
run_TC = False
if run_TC:
    sql_mp_no_outg_TC = (
    """
    SELECT {}
    FROM default.meter_premise
    WHERE state_cd IN ('OH')
    AND trsf_pole_nb NOT IN ({})
    """
    ).format(','.join(cols_of_interest_met_prem), 
             ','.join(["'{}'".format(x) for x in df_outage_location_ids['LOCATION_ID'].tolist()]))
    #print(sql_mp_no_outg_TC)
    #------------------------
    df_mp_no_outg_TC_OG = pd.read_sql(sql_mp_no_outg, conn_aws) 
    print(df_mp_no_outg_TC_OG.shape)
    df_mp_no_outg_TC = df_mp_no_outg_TC_OG.copy()    
    #------------------------
    df_mp_no_outg_TC=df_mp_no_outg_TC.sort_values(by=['trsf_pole_nb', 'prem_nb', 'mfr_devc_ser_nbr'], ignore_index=True)
    df_mp_no_outg=df_mp_no_outg.sort_values(by=['trsf_pole_nb', 'prem_nb', 'mfr_devc_ser_nbr'], ignore_index=True)
    print(df_mp_no_outg_TC.equals(df_mp_no_outg))    

In [21]:
df_mp_no_outg = Utilities_df.convert_col_types(
    df=df_mp_no_outg, 
    cols_and_types_dict={'annual_kwh':float, 'annual_max_dmnd':float}, 
    to_numeric_errors='coerce', 
    inplace=True
)

# ---------------------------------------------------------------
# OUTAGES
# ---------------------------------------------------------------

In [ ]:
def get_sql_outage_0(date_0, date_1, mjr_mnr_cause):
    if mjr_mnr_cause is None:
        add_where_statement_1 = ''
        add_where_statement_2 = ''
    else:
        add_where_statement_1 = f"AND  DOV.MJR_CAUSE_CD = '{mjr_mnr_cause[0]}'"
        add_where_statement_2 = f"AND  DOV.MNR_CAUSE_CD = '{mjr_mnr_cause[1]}'"
    
    sql_outage = (
    """
    SELECT   DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, 
             DOV.DT_ON_TS, DOV.DT_OFF_TS, 
             DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24) AS DT_OFF_TS_FULL, DOV.STEP_DRTN_NB,
             EXTRACT(YEAR FROM DOV.DT_OFF_TS)  AS START_YEAR,     
             DOV.MJR_CAUSE_CD, DOV.MNR_CAUSE_CD, DOV4.EQUIP_TYP_NM, DOV4.SHORT_NM, 
             DOV.LOCATION_ID, DOV.GIS_CRCT_NB, DOV.STATE_ABBR_TX, 
             PRIM.PREMISE_NB
    FROM     DOVSADM.DOVS_OUTAGE_FACT DOV
             LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
             LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
             LEFT OUTER JOIN DOVSADM.DOVS_EQUIPMENT_TYPES_DIM DOV4 ON DOV.EQUIPMENT_CD=DOV4.EQUIPMENT_CD
             LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_CAUSE_TYPES_DIM DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
             LEFT OUTER JOIN DOVSADM.DOVS_PREMISE_DIM PRIM ON DOV.OUTG_REC_NB=PRIM.OUTG_REC_NB 
    WHERE    DOV.DT_OFF_TS BETWEEN {} AND '{}' {}{}
    AND  DOV2.INTRPTN_TYP_CD = 'S'
    AND  DOV2.CURR_REC_STAT_CD = 'A' 
    AND  DOV.MJR_CAUSE_CD <> 'NI' 
    AND  DOV.DEVICE_CD <> 85 
    AND  DOV.STATE_ABBR_TX = 'OH'
    """
    ).format(f"'{date_0}'", date_1, add_where_statement_1, add_where_statement_2)

    return sql_outage

In [ ]:
sql_outage_full = DOVSOutages_SQL.build_sql_std_outage(mjr_mnr_cause=None, 
                                                       include_premise=True, 
                                                       date_range=[date_0_outg, date_1_outg], 
                                                       states='OH').get_sql_statement()
sql_outage_dl_eqf = DOVSOutages_SQL.build_sql_std_outage(mjr_mnr_cause=('DL', 'EQF'), 
                                                         include_premise=True, 
                                                         date_range=[date_0_outg, date_1_outg], 
                                                         states='OH').get_sql_statement()
sql_outage_dl_ol = DOVSOutages_SQL.build_sql_std_outage(mjr_mnr_cause=('DL', 'OL'), 
                                                        include_premise=True, 
                                                        date_range=[date_0_outg, date_1_outg], states='OH').get_sql_statement()
print(sql_outage_full)

In [ ]:
compare_to_og = False
if compare_to_og:
    if 'conn_outages' not in locals():
        conn_outages = Utilities.get_utldb01p_oracle_connection()  
    #---------------------------
    print('dl_ol')
    df_outage     = pd.read_sql_query(sql_outage_dl_ol, conn_outages, dtype={'CI_NB':np.int32, 'CMI_NB':np.float, 'OUTG_REC_NB':np.int32})
    sql_outage_dl_ol_0  = get_sql_outage_0(date_0_outg, date_1_outg, ('DL', 'OL'))
    df_outage_0   = pd.read_sql_query(sql_outage_dl_ol_0, conn_outages, dtype={'CI_NB':np.int32, 'CMI_NB':np.float, 'OUTG_REC_NB':np.int32})
    cols_in_new_not_old = [x for x in df_outage.columns if x not in df_outage_0.columns]
    print(f'FYI: new columns added:\n{cols_in_new_not_old}')
    print(df_outage[df_outage_0.columns].sort_values(by=['OUTG_REC_NB', 'PREMISE_NB'], ignore_index=True).equals(df_outage_0.sort_values(by=['OUTG_REC_NB', 'PREMISE_NB'], ignore_index=True)))
    print()
    #---------------------------
    print('dl_eqf')
    df_outage    = pd.read_sql_query(sql_outage_dl_eqf, conn_outages, dtype={'CI_NB':np.int32, 'CMI_NB':np.float, 'OUTG_REC_NB':np.int32})
    sql_outage_dl_eqf_0 = get_sql_outage_0(date_0_outg, date_1_outg, ('DL', 'EQF'))
    df_outage_0  = pd.read_sql_query(sql_outage_dl_eqf_0, conn_outages, dtype={'CI_NB':np.int32, 'CMI_NB':np.float, 'OUTG_REC_NB':np.int32})
    cols_in_new_not_old = [x for x in df_outage.columns if x not in df_outage_0.columns]
    print(f'FYI: new columns added:\n{cols_in_new_not_old}')
    print(df_outage[df_outage_0.columns].sort_values(by=['OUTG_REC_NB', 'PREMISE_NB'], ignore_index=True).equals(df_outage_0.sort_values(by=['OUTG_REC_NB', 'PREMISE_NB'], ignore_index=True)))
    print()
    #---------------------------
    print('full')
    df_outage      = pd.read_sql_query(sql_outage_full, conn_outages, dtype={'CI_NB':np.int32, 'CMI_NB':np.float, 'OUTG_REC_NB':np.int32})
    sql_outage_full_0   = get_sql_outage_0(date_0_outg, date_1_outg, None)
    df_outage_0    = pd.read_sql_query(sql_outage_full_0, conn_outages, dtype={'CI_NB':np.int32, 'CMI_NB':np.float, 'OUTG_REC_NB':np.int32})
    cols_in_new_not_old = [x for x in df_outage.columns if x not in df_outage_0.columns]
    print(f'FYI: new columns added:\n{cols_in_new_not_old}')
    print(df_outage[df_outage_0.columns].sort_values(by=['OUTG_REC_NB', 'PREMISE_NB'], ignore_index=True).equals(df_outage_0.sort_values(by=['OUTG_REC_NB', 'PREMISE_NB'], ignore_index=True)))
    print()

In [ ]:
if read_dfs_from_csv:
    df_outage_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_outage_OG.csv'), dtype=str)
    csv_cols_and_types_to_convert_dict = {'CI_NB':np.int32, 'CMI_NB':np.float, 'OUTG_REC_NB':[np.float, np.int32]}
    df_outage_OG = Utilities_df.convert_col_types(df_outage_OG, csv_cols_and_types_to_convert_dict)
else:
    df_outage_OG = pd.read_sql_query(sql_outage_full, conn_outages, dtype={'CI_NB':np.int32, 
                                                                           'CMI_NB':np.float, 
                                                                           'OUTG_REC_NB':np.int32})

# Save CSVs if save_dfs_to_csv = True

In [ ]:
if save_dfs_to_csv:
    df_outage_location_ids_OG.to_csv(os.path.join(save_dir_csvs, 'df_outage_location_ids_OG.csv'), index=False)
    df_mp_all_OG.to_csv(os.path.join(save_dir_csvs, 'df_mp_all_OG.csv'), index=False)
    df_mp_no_outg_OG.to_csv(os.path.join(save_dir_csvs, 'df_mp_no_outg_OG.csv'), index=False)
    df_outage_OG.to_csv(os.path.join(save_dir_csvs, 'df_outage_OG.csv'), index=False)

# Other functions

# Merge df_outage with df_mp_all and build subsets (e.g., df_outage_prim, df_outage_dl_eqf, etc.) 

In [ ]:
# Join df_outage with df_mp_all to append mfr_devc_ser_nbr and trsf_pole_nb data.
#-------------------------
# Remember, in general a premise can have mutliple meters.  Therefore, one should expect
#   the resultant dataframe to be larger than the original.
# This default method is:
#       df_outage = df_outage.merge(df_mp_all[['mfr_devc_ser_nbr', 'prem_nb', 'trsf_pole_nb']], how='left', 
#                                   left_on='PREMISE_NB', right_on='prem_nb')
#-------------------------
# If, instead, one wants the resultant dataframe to be the same size as the original, the
#   mfr_devc_ser_nbr data should not be included.
#   This can be achieved e.g., by:
#         df_outage = df_outage.merge(df_mp_all[['prem_nb', 'trsf_pole_nb']].drop_duplicates(), how='left', 
#                                     left_on='PREMISE_NB', right_on='prem_nb')
#-------------------------
# NOTE: Cases where df_outage['PREMISE_NB']!=df_outage['prem_nb'] should only occur when
#       df_outage['prem_nb'] is NaN, meaning the premise number from the outage data
#       was not found in the meter premise

In [ ]:
#TODO why are prem_nb in df_mp_all all integers?  Whereas PREMISE_NB in df_outage
# are strings, and can be purely numerical, a alphanumeric, or all letters
if read_dfs_from_csv:
    df_mp_all['prem_nb'] = df_mp_all['prem_nb'].astype(str)

In [ ]:
df_outage = df_outage_OG.copy()
df_outage = df_outage.merge(df_mp_all, how='left', 
                            left_on='PREMISE_NB', right_on='prem_nb')
df_outage = Utilities_df.convert_col_types(
    df=df_outage, 
    cols_and_types_dict={'annual_kwh':float, 'annual_max_dmnd':float}, 
    to_numeric_errors='coerce', 
    inplace=True
)
# Below, 'prim' stands for primary, and means the meters are connected directly to a transformer pole causing an outage
df_outage_prim = df_outage[df_outage['LOCATION_ID']==df_outage['trsf_pole_nb']].copy()

# Below, 'prim_strict' stands for primary strict, and means the  meters are connected directly 
# to a transformer pole causing an outage, AND the equipment type causing the outage is a transformer
# (exact EQUIP_TYP_NMs given in xfmr_equip_typ_nms_of_interest)
xfmr_equip_typ_nms_of_interest = ['TRANSFORMER, OH', 'TRANSFORMER, UG']
df_outage_prim_strict = df_outage[(df_outage['LOCATION_ID']==df_outage['trsf_pole_nb']) & 
                                  (df_outage['EQUIP_TYP_NM'].isin(xfmr_equip_typ_nms_of_interest))].copy()

In [ ]:
df_outage[df_outage['PREMISE_NB'] != df_outage['prem_nb']]

In [ ]:
df_outage.shape

# Find top outage causes 

In [ ]:
# Grab top 5 outage causes
# Just want one outage per column without repeat
# To achieve this, utilize cols_for_drop_duplicates
cols_for_drop_duplicates = ['CI_NB', 'CMI_NB', 'OUTG_REC_NB', 'STATE_ABBR_TX', 'MJR_CAUSE_CD', 'MNR_CAUSE_CD', 'LOCATION_ID']
top_n=5
top_outage_metrics=['cmi', 'ci', 'n_outages']
mjr_cause_col='MJR_CAUSE_CD'
mnr_cause_col='MNR_CAUSE_CD'
cmi_nb_col='CMI_NB'
ci_nb_col='CI_NB'

top_5_outage_mjr_mnr_causes = DOVSOutages.get_top_n_outage_mjr_mnr_causes(df_outage, cols_for_drop_duplicates, 
                                                              top_outage_metrics=top_outage_metrics, top_n=top_n, 
                                                              mjr_cause_col=mjr_cause_col, mnr_cause_col=mnr_cause_col, 
                                                              cmi_nb_col=cmi_nb_col, ci_nb_col=ci_nb_col)
#---------------------------------------
top5_mjr_mnr_causes_ci = top_5_outage_mjr_mnr_causes['ci']
top5_mjr_mnr_causes_cmi = top_5_outage_mjr_mnr_causes['cmi']
top5_mjr_mnr_causes_count = top_5_outage_mjr_mnr_causes['n_outages']
#---------------------------------------
# Build subset dfs
(top5_df_outages_ci, 
 top5_df_outages_by_xfmr_ci) = DOVSOutages.build_top_5_df_outages_subsets(top5_mjr_mnr_causes_ci, df_outage)
(top5_df_outages_prim_ci, 
 top5_df_outages_prim_by_xfmr_ci) = DOVSOutages.build_top_5_df_outages_subsets(top5_mjr_mnr_causes_ci, df_outage_prim)
(top5_df_outages_prim_strict_ci, 
 top5_df_outages_prim_strict_by_xfmr_ci) = DOVSOutages.build_top_5_df_outages_subsets(top5_mjr_mnr_causes_ci, df_outage_prim_strict)
#-----
(top5_df_outages_cmi, 
 top5_df_outages_by_xfmr_cmi) = DOVSOutages.build_top_5_df_outages_subsets(top5_mjr_mnr_causes_cmi, df_outage)
(top5_df_outages_prim_cmi, 
 top5_df_outages_prim_by_xfmr_cmi) = DOVSOutages.build_top_5_df_outages_subsets(top5_mjr_mnr_causes_cmi, df_outage_prim)
(top5_df_outages_prim_strict_cmi, 
 top5_df_outages_prim_strict_by_xfmr_cmi) = DOVSOutages.build_top_5_df_outages_subsets(top5_mjr_mnr_causes_cmi, df_outage_prim_strict)
#-----
(top5_df_outages_count, 
 top5_df_outages_by_xfmr_count) = DOVSOutages.build_top_5_df_outages_subsets(top5_mjr_mnr_causes_count, df_outage)
(top5_df_outages_prim_count, 
 top5_df_outages_prim_by_xfmr_count) = DOVSOutages.build_top_5_df_outages_subsets(top5_mjr_mnr_causes_count, df_outage_prim)
(top5_df_outages_prim_strict_count, 
 top5_df_outages_prim_strict_by_xfmr_count) = DOVSOutages.build_top_5_df_outages_subsets(top5_mjr_mnr_causes_count, df_outage_prim_strict)

In [ ]:
#TODO NEED TO ADJUST DOVSOutages.consolidate_df_outage now that e.g. mfr_devc_ser_nbr is included in df_outage
# df_outage_consolidate = DOVSOutages.consolidate_df_outage(df_outage, cols_to_drop=None)
# df_outage_consolidate.head()

In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# NOTE: Duplicates need to be removed for plotting
#   e.g., df1_outg_and_mp_prem[[x for x in df1_outg_and_mp_prem.columns if x in cols_of_interest_met_prem]].drop_duplicates()
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Build meter premise DataFrame

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# NOTE: If multiple serial number exist for a premise number, it appear (UPON FIRST, LIMITED INSPECTION)
# that all values are the same between the two serial number EXCEPT mfr_devc_ser_nbr (obviously)
# and possibly longitude,latitude
# This means, e.g. annual_kwh and annual_max_dmnd are the same for the two serial numbers
#   Which implies there are actually tallied per premise, not per meter
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

I used to run targeted SQL queries.  
However, it is actually MUCH quicker to first build df_mp_all (consisting of all the meter premise data for OH),
and then select the outage subset from it.
Furthermore, df_mp_all is now built above, so it's ready to go.
The old, targeted, SQL queries (sql_mp_full_outg and sql_mp) are reproduced below, but no longer used

In [ ]:
sql_mp_full_outg = (
"""
SELECT {}
FROM default.meter_premise
WHERE state_cd IN ({})
AND prem_nb IN ({})
"""
).format(','.join(cols_of_interest_met_prem), 
         ','.join(["'{}'".format(x) for x in df_outage['STATE_ABBR_TX'].unique().tolist()]), 
         ','.join(["'{}'".format(x) for x in df_outage['PREMISE_NB'].unique().tolist()]))
#print(sql_mp_full_outg)

In [ ]:
sql_mp = (
"""
SELECT {}
FROM default.meter_premise
WHERE state_cd IN ({})
AND trsf_pole_nb IN ({})
"""
).format(','.join(cols_of_interest_met_prem), 
         ','.join(["'{}'".format(x) for x in df_outage['STATE_ABBR_TX'].unique().tolist()]), 
         ','.join(["'{}'".format(x) for x in df_outage['LOCATION_ID'].unique().tolist()]))
#print(sql_mp)

In [ ]:
df_outage_save = df_outage.copy()
df_outage_prim_save = df_outage_prim.copy()
df_outage_prim_strict_save = df_outage_prim_strict.copy()

### Keep only entries whose trsf_pole_nb contains at least one numerical digit
Eliminates, e.g. ['TRANSMISSION', 'NETWORK', ' ', 'PRIMARY', 'IDONOTKNOWXX','HJKHKJHKJ']

In [ ]:
df_outage = df_outage_save.copy()
df_outage_prim = df_outage_prim_save.copy()
df_outage_prim_strict = df_outage_prim_strict_save.copy()

In [ ]:
[df_outage, 
 df_outage_prim, 
 df_outage_prim_strict, 
 df_mp_no_outg] = DOVSOutages.remove_trsf_pole_nb_without_numeric_digit([df_outage, 
                                                            df_outage_prim, 
                                                            df_outage_prim_strict, 
                                                            df_mp_no_outg])
#---------------------------------
df_outage_dl_eqf = DOVSOutages.get_mjr_mnr_cause_df_outage_subset(df_outage, 'DL', 'EQF', return_copy=True)
df_outage_dl_eqf_prim = DOVSOutages.get_mjr_mnr_cause_df_outage_subset(df_outage_prim, 'DL', 'EQF', return_copy=True)
df_outage_dl_eqf_prim_strict = DOVSOutages.get_mjr_mnr_cause_df_outage_subset(df_outage_prim_strict, 'DL', 'EQF', return_copy=True)

df_outage_dl_ol = DOVSOutages.get_mjr_mnr_cause_df_outage_subset(df_outage, 'DL', 'OL', return_copy=True)
df_outage_dl_ol_prim = DOVSOutages.get_mjr_mnr_cause_df_outage_subset(df_outage_prim, 'DL', 'OL', return_copy=True)
df_outage_dl_ol_prim_strict = DOVSOutages.get_mjr_mnr_cause_df_outage_subset(df_outage_prim_strict, 'DL', 'OL', return_copy=True)

In [ ]:
df_outage.shape

In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# NOTE: Duplicates need to be removed for plotting
#   e.g., df1_outg_and_mp_prem[[x for x in df1_outg_and_mp_prem.columns if x in cols_of_interest_met_prem]].drop_duplicates()
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
# FUCK
# TODO develop method which takes in df_outage type dataframe and output
# df_outage_by_meter, df_outage_by_xfmr, df_outage_by_outage
#
# TODO develop method which takes in df_outage type dataframe and outputs
# df_outage_prim, df_outage_prim_strict
#
# TODO combine two methods

In [ ]:
# By meter = Aggregate for entire pool of meters.  
#            Each meter counted only once, regardless of the number of outages suffered
# By transformer = For each transformer in the collection, all connected meters are first aggregated
#                  All transformers in the collection are then aggregated
#                  Each transformer contributes once, regardless of the number of outages suffered
# By outage = Similar to by transformer, but each transformer contributes as many times as outages it suffered
#-------------------------
# by_mxo = by_meter_xfmr_outage
#---------------------------------------------------------
cols_for_drop_duplicates = [x for x in df_outage.columns if x in cols_of_interest_met_prem]
#-------------------------
dfs_outg_dict_by_mxo = DOVSOutages.build_df_outage_by_meter_xfmr_outage(df_outage, cols_for_drop_duplicates)
#-------------------------
dfs_outg_dict_by_mxo_prim = DOVSOutages.build_df_outage_by_meter_xfmr_outage(df_outage_prim, cols_for_drop_duplicates)
#-------------------------
dfs_outg_dict_by_mxo_prim_strict = DOVSOutages.build_df_outage_by_meter_xfmr_outage(df_outage_prim_strict, cols_for_drop_duplicates)
#-------------------------
#---------------------------------------------------------
dfs_outg_dict_by_mxo_dl_ol = DOVSOutages.build_df_outage_by_meter_xfmr_outage(DOVSOutages.get_mjr_mnr_cause_df_outage_subset(df_outage, 'DL', 'OL', return_copy=True), 
                                                                  cols_for_drop_duplicates)
#-------------------------
dfs_outg_dict_by_mxo_dl_ol_prim = DOVSOutages.build_df_outage_by_meter_xfmr_outage(DOVSOutages.get_mjr_mnr_cause_df_outage_subset(df_outage_prim, 'DL', 'OL', return_copy=True), 
                                                                  cols_for_drop_duplicates)
#-------------------------
dfs_outg_dict_by_mxo_dl_ol_prim_strict = DOVSOutages.build_df_outage_by_meter_xfmr_outage(DOVSOutages.get_mjr_mnr_cause_df_outage_subset(df_outage_prim_strict, 'DL', 'OL', return_copy=True), 
                                                                  cols_for_drop_duplicates)
#-------------------------
#---------------------------------------------------------
dfs_outg_dict_by_mxo_dl_eqf = DOVSOutages.build_df_outage_by_meter_xfmr_outage(DOVSOutages.get_mjr_mnr_cause_df_outage_subset(df_outage, 'DL', 'EQF', return_copy=True), 
                                                                   cols_for_drop_duplicates)
#-------------------------
dfs_outg_dict_by_mxo_dl_eqf_prim = DOVSOutages.build_df_outage_by_meter_xfmr_outage(DOVSOutages.get_mjr_mnr_cause_df_outage_subset(df_outage_prim, 'DL', 'EQF', return_copy=True), 
                                                                        cols_for_drop_duplicates)
#-------------------------
dfs_outg_dict_by_mxo_dl_eqf_prim_strict = DOVSOutages.build_df_outage_by_meter_xfmr_outage(DOVSOutages.get_mjr_mnr_cause_df_outage_subset(df_outage_prim_strict, 'DL', 'EQF', return_copy=True), 
                                                                        cols_for_drop_duplicates)
#-------------------------
#---------------------------------------------------------

In [ ]:
# df_mp_no_outg
# By meter
# Not really necessary for df_mp_no_outg, as the call below does nothing
df_mp_no_outg_by_meter = df_mp_no_outg[[x for x in df_mp_no_outg.columns if x in cols_of_interest_met_prem]].drop_duplicates().dropna(how='all')
assert (df_mp_no_outg.equals(df_mp_no_outg_by_meter))
# By transformer
df_mp_no_outg_by_xfmr  = df_mp_no_outg_by_meter.groupby('trsf_pole_nb')[['annual_kwh', 'annual_max_dmnd']].sum()

# by_mx = by_meter_xfmr
dfs_mp_no_outg_dict_by_mx = {'by_meter':df_mp_no_outg_by_meter, 
                             'by_xfmr':df_mp_no_outg_by_xfmr}

In [ ]:
print(f"df_mp_all_OG.shape            = {df_mp_all_OG.shape}")
print(f"df_mp_all.shape               = {df_mp_all.shape}")
print()
print(f"df_outage_OG.shape            = {df_outage_OG.shape}")
print(f"df_outage.shape               = {df_outage.shape}")
print()
print(f"dfs_outg_dict_by_mxo_prim['by_meter'].shape = {dfs_outg_dict_by_mxo_prim['by_meter'].shape}")
print(f"dfs_outg_dict_by_mxo_prim['by_xfmr'].shape  = {dfs_outg_dict_by_mxo_prim['by_xfmr'].shape}")
print()

In [ ]:
# sns.histplot(data=tips, y="size", color = "green")
dfs_outg_dict_by_mxo_prim_strict['by_xfmr']['annual_kwh']

# By transformer

In [ ]:
conn_outages = Utilities.get_utldb01p_oracle_connection()
df_outg_cause_dim = pd.read_sql('SELECT * FROM DOVSADM.DOVS_OUTAGE_CAUSE_TYPES_DIM', conn_outages)  

In [ ]:
min_max_and_bin_size=(0.0, 2.5e5, 2.5e4/2.0)
stat='density'
include_over_underflow=[True, False]
keep_edges_opaque=True
include_hatches=True
draw_side_by_side=True
draw_single_idx_full_width=3
alpha=0.250
#----------------------------------------
fig, (ax0,ax1,ax2) = plt.subplots(3, 1, num=fig_num, figsize=[14, 18])
subplots_adjust_args = get_subplots_adjust_args_annual_kwh_3x1()
plt.subplots_adjust(**subplots_adjust_args)
#----------------------------------------
dfs_with_args = [(dfs_outg_dict_by_mxo_prim_strict['by_xfmr'], dict(color='goldenrod', alpha=alpha, label="Outages (Direct)")), 
                 (dfs_outg_dict_by_mxo_prim['by_xfmr'], dict(color='green', alpha=alpha, label="Outages (Xfmr tied to Outg. Location ID)")), 
                 (dfs_outg_dict_by_mxo['by_xfmr'],      dict(color='red', alpha=alpha, label="Outages (All Xfmrs)")), 
                 (dfs_mp_no_outg_dict_by_mx['by_xfmr'], dict(color='blue', hatch=None, alpha=alpha/2, label="No Outages", zorder=-1))]
ax0 = plot_multiple_annual_kwh_hist(ax0, dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=f'Annual kWh - Transformer Level (All Outage Causes)', fontsize=20), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                              dict(axis='y', labelsize=15)])
#----------------------------------------
dfs_with_args = [(dfs_outg_dict_by_mxo_dl_ol_prim_strict['by_xfmr'], dict(color='goldenrod', alpha=alpha, label="Outages (Direct)")), 
                 (dfs_outg_dict_by_mxo_dl_ol_prim['by_xfmr'], dict(color='green', alpha=alpha, label="Outages (Xfmr tied to Outg. Location ID)")),          
                 (dfs_outg_dict_by_mxo_dl_ol['by_xfmr'],      dict(color='red', alpha=alpha, label="Outages (All Xfmrs)")), 
                 (dfs_mp_no_outg_dict_by_mx['by_xfmr'],       dict(color='blue', hatch=None, alpha=alpha/2.0, label="No Outages"))]
title = f"Annual kWh - Transformer Level ({get_mjr_mnr_nms_from_cds(('DL', 'OL'), df_outg_cause_dim, combine_output=True)})"
ax1 = plot_multiple_annual_kwh_hist(ax1, dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=title, fontsize=20), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                               dict(axis='y', labelsize=15)])
#----------------------------------------
dfs_with_args = [(dfs_outg_dict_by_mxo_dl_eqf_prim_strict['by_xfmr'], dict(color='goldenrod', alpha=alpha, label="Outages (Direct)")), 
                 (dfs_outg_dict_by_mxo_dl_eqf_prim['by_xfmr'], dict(color='green', alpha=alpha, label="Outages (Xfmr tied to Outg. Location ID)")),  
                 (dfs_outg_dict_by_mxo_dl_eqf['by_xfmr'],      dict(color='red', alpha=alpha, label="Outages (All Xfmrs)")), 
                 (dfs_mp_no_outg_dict_by_mx['by_xfmr'],        dict(color='blue', alpha=alpha/2, hatch=None, label="No Outages"))]
title = f"Annual kWh - Transformer Level ({get_mjr_mnr_nms_from_cds(('DL', 'EQF'), df_outg_cause_dim, combine_output=True)})"
ax2 = plot_multiple_annual_kwh_hist(ax2, dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=title, fontsize=20), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                               dict(axis='y', labelsize=15)])
#----------------------------------------
# if save_figs:
#     save_dir = os.path.join(save_dir_base_figs , 'annual_kwh', 'XfmrLevel', 'PDF')
#     save_name = 'annual_kwh_hist_direct_vs_all_3x1_causes.pdf'
#     save_fig(fig, save_dir, save_name)
#     #-----
#     save_dir = os.path.join(save_dir_base_figs , 'annual_kwh', 'XfmrLevel', 'PNG')
#     save_name = 'annual_kwh_hist_direct_vs_all_3x1_causes.png'
#     save_fig(fig, save_dir, save_name)
#----------------------------------------
fig_num+=1

In [ ]:
min_max_and_bin_size=(0.0, 2.5e5, 2.5e4/2.0)
stat='density'
include_over_underflow=[True, False]
keep_edges_opaque=True
include_hatches=True
draw_side_by_side=True
draw_single_idx_full_width=3
alpha=0.250
#----------------------------------------
fig, (ax0,ax1,ax2) = plt.subplots(3, 1, num=fig_num, figsize=[14, 18])
subplots_adjust_args = get_subplots_adjust_args_annual_kwh_3x1()
plt.subplots_adjust(**subplots_adjust_args)
#----------------------------------------
dfs_with_args = [(dfs_outg_dict_by_mxo_prim_strict['by_xfmr'], dict(color='goldenrod', alpha=alpha, label="Outages (Direct)")), 
                 (dfs_outg_dict_by_mxo_prim['by_xfmr'], dict(color='green', alpha=alpha, label="Outages (Xfmr tied to Outg. Location ID)")), 
                 (dfs_outg_dict_by_mxo['by_xfmr'],      dict(color='red', alpha=alpha, label="Outages (All Xfmrs)")), 
                 (dfs_mp_no_outg_dict_by_mx['by_xfmr'], dict(color='blue', hatch=None, alpha=alpha/2, label="No Outages", zorder=-1))]
ax0 = plot_multiple_annual_kwh_hist(ax0, dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    orient='h', 
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=f'Annual kWh - Transformer Level (All Outage Causes)', fontsize=20), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                              dict(axis='y', labelsize=15)])
#----------------------------------------
dfs_with_args = [(dfs_outg_dict_by_mxo_dl_ol_prim_strict['by_xfmr'], dict(color='goldenrod', alpha=alpha, label="Outages (Direct)")), 
                 (dfs_outg_dict_by_mxo_dl_ol_prim['by_xfmr'], dict(color='green', alpha=alpha, label="Outages (Xfmr tied to Outg. Location ID)")),          
                 (dfs_outg_dict_by_mxo_dl_ol['by_xfmr'],      dict(color='red', alpha=alpha, label="Outages (All Xfmrs)")), 
                 (dfs_mp_no_outg_dict_by_mx['by_xfmr'],       dict(color='blue', hatch=None, alpha=alpha/2.0, label="No Outages"))]
title = f"Annual kWh - Transformer Level ({get_mjr_mnr_nms_from_cds(('DL', 'OL'), df_outg_cause_dim, combine_output=True)})"
ax1 = plot_multiple_annual_kwh_hist(ax1, dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    orient='h', 
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=title, fontsize=20), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                               dict(axis='y', labelsize=15)])
#----------------------------------------
dfs_with_args = [(dfs_outg_dict_by_mxo_dl_eqf_prim_strict['by_xfmr'], dict(color='goldenrod', alpha=alpha, label="Outages (Direct)")), 
                 (dfs_outg_dict_by_mxo_dl_eqf_prim['by_xfmr'], dict(color='green', alpha=alpha, label="Outages (Xfmr tied to Outg. Location ID)")),  
                 (dfs_outg_dict_by_mxo_dl_eqf['by_xfmr'],      dict(color='red', alpha=alpha, label="Outages (All Xfmrs)")), 
                 (dfs_mp_no_outg_dict_by_mx['by_xfmr'],        dict(color='blue', alpha=alpha/2, hatch=None, label="No Outages"))]
title = f"Annual kWh - Transformer Level ({get_mjr_mnr_nms_from_cds(('DL', 'EQF'), df_outg_cause_dim, combine_output=True)})"
ax2 = plot_multiple_annual_kwh_hist(ax2, dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    orient='h', 
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=title, fontsize=20), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                               dict(axis='y', labelsize=15)])
#----------------------------------------
# if save_figs:
#     save_dir = os.path.join(save_dir_base_figs , 'annual_kwh', 'XfmrLevel', 'PDF')
#     save_name = 'annual_kwh_hist_direct_vs_all_3x1_causes.pdf'
#     save_fig(fig, save_dir, save_name)
#     #-----
#     save_dir = os.path.join(save_dir_base_figs , 'annual_kwh', 'XfmrLevel', 'PNG')
#     save_name = 'annual_kwh_hist_direct_vs_all_3x1_causes.png'
#     save_fig(fig, save_dir, save_name)
#----------------------------------------
fig_num+=1

In [ ]:
min_max_and_bin_size=(0.0, 2.5e5, 2.5e4/2.0)
stat='density'
include_over_underflow=[True, False]
keep_edges_opaque=True
include_hatches=True
draw_side_by_side=True
draw_single_idx_full_width=3
alpha=0.250
#----------------------------------------
fig, (ax0,ax1,ax2) = plt.subplots(3, 1, num=fig_num, figsize=[14, 18])
subplots_adjust_args = get_subplots_adjust_args_annual_kwh_3x1()
plt.subplots_adjust(**subplots_adjust_args)
#----------------------------------------
dfs_with_args = [(dfs_outg_dict_by_mxo_prim_strict['by_xfmr'], dict(color='goldenrod', alpha=alpha, label="Outages (Direct)")), 
                 (dfs_outg_dict_by_mxo_prim['by_xfmr'], dict(color='green', alpha=alpha, label="Outages (Xfmr tied to Outg. Location ID)")), 
                 (dfs_outg_dict_by_mxo['by_xfmr'],      dict(color='red', alpha=alpha, label="Outages (All Xfmrs)")), 
                 (dfs_mp_no_outg_dict_by_mx['by_xfmr'], dict(color='blue', hatch=None, alpha=alpha/2, label="No Outages", zorder=-1))]
ax0 = plot_multiple_annual_kwh_hist(ax0, dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=True, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    orient='h', 
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=f'Annual kWh - Transformer Level (All Outage Causes)', fontsize=20), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                              dict(axis='y', labelsize=15)])
#----------------------------------------
dfs_with_args = [(dfs_outg_dict_by_mxo_dl_ol_prim_strict['by_xfmr'], dict(color='goldenrod', alpha=alpha, label="Outages (Direct)")), 
                 (dfs_outg_dict_by_mxo_dl_ol_prim['by_xfmr'], dict(color='green', alpha=alpha, label="Outages (Xfmr tied to Outg. Location ID)")),          
                 (dfs_outg_dict_by_mxo_dl_ol['by_xfmr'],      dict(color='red', alpha=alpha, label="Outages (All Xfmrs)")), 
                 (dfs_mp_no_outg_dict_by_mx['by_xfmr'],       dict(color='blue', hatch=None, alpha=alpha/2.0, label="No Outages"))]
title = f"Annual kWh - Transformer Level ({get_mjr_mnr_nms_from_cds(('DL', 'OL'), df_outg_cause_dim, combine_output=True)})"
ax1 = plot_multiple_annual_kwh_hist(ax1, dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=True, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    orient='h', 
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=title, fontsize=20), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                               dict(axis='y', labelsize=15)])
#----------------------------------------
dfs_with_args = [(dfs_outg_dict_by_mxo_dl_eqf_prim_strict['by_xfmr'], dict(color='goldenrod', alpha=alpha, label="Outages (Direct)")), 
                 (dfs_outg_dict_by_mxo_dl_eqf_prim['by_xfmr'], dict(color='green', alpha=alpha, label="Outages (Xfmr tied to Outg. Location ID)")),  
                 (dfs_outg_dict_by_mxo_dl_eqf['by_xfmr'],      dict(color='red', alpha=alpha, label="Outages (All Xfmrs)")), 
                 (dfs_mp_no_outg_dict_by_mx['by_xfmr'],        dict(color='blue', alpha=alpha/2, hatch=None, label="No Outages"))]
title = f"Annual kWh - Transformer Level ({get_mjr_mnr_nms_from_cds(('DL', 'EQF'), df_outg_cause_dim, combine_output=True)})"
ax2 = plot_multiple_annual_kwh_hist(ax2, dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=True, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    orient='h', 
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=title, fontsize=20), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                               dict(axis='y', labelsize=15)])
#----------------------------------------
# if save_figs:
#     save_dir = os.path.join(save_dir_base_figs , 'annual_kwh', 'XfmrLevel', 'PDF')
#     save_name = 'annual_kwh_hist_direct_vs_all_3x1_causes.pdf'
#     save_fig(fig, save_dir, save_name)
#     #-----
#     save_dir = os.path.join(save_dir_base_figs , 'annual_kwh', 'XfmrLevel', 'PNG')
#     save_name = 'annual_kwh_hist_direct_vs_all_3x1_causes.png'
#     save_fig(fig, save_dir, save_name)
#----------------------------------------
fig_num+=1

In [ ]:
dfs_dicts_with_args_test = {'dfs_outg_dict_by_mxo_prim':
                       (dfs_outg_dict_by_mxo_prim,        dict(color='green', alpha=alpha, label="Outages (Xfmr tied to Outg. Location ID)")), 
                       'dfs_outg_dict_by_mxo_prim_strict':
                       (dfs_outg_dict_by_mxo_prim_strict, dict(color='goldenrod', alpha=alpha, label="Outages (Direct)")), 
                       'dfs_outg_dict_by_mxo':
                       (dfs_outg_dict_by_mxo,             dict(color='red', alpha=alpha, label="Outages (All Xfmrs)")), 
                       'dfs_mp_no_outg_dict_by_mx':
                       (dfs_mp_no_outg_dict_by_mx,        dict(color='blue', hatch=None, alpha=alpha/2.0, label="No Outages", zorder=-1))}

In [ ]:
min_max_and_bin_size=(0.0, 2.5e5, 2.5e4/2.0)
stat='density'
include_over_underflow=[True, False]
keep_edges_opaque=True
include_hatches=True
draw_side_by_side=True
draw_single_idx_full_width=3
alpha=0.250
#----------------------------------------
fig, (ax0,ax1,ax2) = plt.subplots(3, 1, num=fig_num, figsize=[14, 18])
subplots_adjust_args = get_subplots_adjust_args_annual_kwh_3x1()
plt.subplots_adjust(**subplots_adjust_args)
#----------------------------------------
dfs_to_include = ['dfs_outg_dict_by_mxo_prim', 'dfs_outg_dict_by_mxo_prim_strict', 'dfs_outg_dict_by_mxo', 'dfs_mp_no_outg_dict_by_mx']
dfs_with_args = [(x[0]['by_xfmr'], x[1]) for k,x in dfs_dicts_with_args_test.items() if k in dfs_to_include]
ax0 = plot_multiple_annual_kwh_hist(ax0, dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=f'Annual kWh - Transformer Level (All Outage Causes)', fontsize=20), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                              dict(axis='y', labelsize=15)])
#----------------------------------------
dfs_with_args = [(dfs_outg_dict_by_mxo_dl_ol_prim['by_xfmr'], dict(color='green', alpha=alpha, label="Outages (Xfmr tied to Outg. Location ID)")), 
                 (dfs_outg_dict_by_mxo_dl_ol_prim_strict['by_xfmr'], dict(color='goldenrod', alpha=alpha, label="Outages (Direct)")), 
                 (dfs_outg_dict_by_mxo_dl_ol['by_xfmr'],      dict(color='red', alpha=alpha, label="Outages (All Xfmrs)")), 
                 (dfs_mp_no_outg_dict_by_mx['by_xfmr'],       dict(color='blue', hatch=None, alpha=alpha/2.0, label="No Outages"))]
title = f"Annual kWh - Transformer Level ({get_mjr_mnr_nms_from_cds(('DL', 'OL'), df_outg_cause_dim, combine_output=True)})"
ax1 = plot_multiple_annual_kwh_hist(ax1, dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=title, fontsize=20), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                               dict(axis='y', labelsize=15)])
#----------------------------------------
dfs_with_args = [(dfs_outg_dict_by_mxo_dl_eqf_prim['by_xfmr'], dict(color='green', alpha=alpha, label="Outages (Xfmr tied to Outg. Location ID)")), 
                 (dfs_outg_dict_by_mxo_dl_eqf_prim_strict['by_xfmr'], dict(color='goldenrod', alpha=alpha, label="Outages (Direct)")), 
                 (dfs_outg_dict_by_mxo_dl_eqf['by_xfmr'],      dict(color='red', alpha=alpha, label="Outages (All Xfmrs)")), 
                 (dfs_mp_no_outg_dict_by_mx['by_xfmr'],        dict(color='blue', alpha=alpha/2, hatch=None, label="No Outages"))]
title = f"Annual kWh - Transformer Level ({get_mjr_mnr_nms_from_cds(('DL', 'EQF'), df_outg_cause_dim, combine_output=True)})"
ax2 = plot_multiple_annual_kwh_hist(ax2, dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=title, fontsize=20), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                               dict(axis='y', labelsize=15)])
#----------------------------------------
# if save_figs:
#     save_dir = os.path.join(save_dir_base_figs , 'annual_kwh', 'XfmrLevel', 'PDF')
#     save_name = 'annual_kwh_hist_direct_vs_all_3x1_causes.pdf'
#     save_fig(fig, save_dir, save_name)
#     #-----
#     save_dir = os.path.join(save_dir_base_figs , 'annual_kwh', 'XfmrLevel', 'PNG')
#     save_name = 'annual_kwh_hist_direct_vs_all_3x1_causes.png'
#     save_fig(fig, save_dir, save_name)
#----------------------------------------
fig_num+=1

# BEGIN DELETE
Why aren't there more counts in by_outage than by_transformer?

In [ ]:
#FUCK TODO
#LOOKING AT PLOT BELOW, WHERE OUTG HAS HUGE OVERFLOW
# ==> Must be a small number of high annual_kwh transformers which 
#     suffer many outages

# TODO: Can I look at number of outages suffered vs annual_kwh
# EVEN BETTER: Can I look at heat map of number of outages with annual_kwh

In [ ]:
min_max_and_bin_size=(0.0, 2.5e5, 2.5e4/2.0)
stat='density'
include_over_underflow=[True, False]
keep_edges_opaque=True
include_hatches=False
draw_side_by_side=True
draw_single_idx_full_width=None
alpha=1.0
#----------------------------------------
fig, axs = plt.subplots(2, 2, num=fig_num, figsize=[28, 12], sharex=False)
axs = axs.flatten()
subplots_adjust_args = {'left':0.075, 'right':0.975, 
                        'bottom':0.075, 'top':0.90, 
                        'wspace':0.2, 'hspace':0.35}
plt.subplots_adjust(**subplots_adjust_args)
#----------------------------------------
dfs_with_args = [(dfs_outg_dict_by_mxo_prim['by_xfmr'],        dict(color='green', alpha=alpha, label="Outages: All Causes (Xfmr Level)")), 
                 (dfs_outg_dict_by_mxo_prim['by_outg'],        dict(color='darkgreen', alpha=alpha, label="Outages: All Causes (Outg Level)"))]
axs[0] = plot_multiple_annual_kwh_hist(axs[0], dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=f'Xfmr tied to Outg. Location ID - Density', fontsize=25), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                               dict(axis='y', labelsize=15)])


dfs_with_args = [(dfs_outg_dict_by_mxo['by_xfmr'],        dict(color='green', alpha=alpha, label="Outages: All Causes (Xfmr Level)")), 
                 (dfs_outg_dict_by_mxo['by_outg'],        dict(color='darkgreen', alpha=alpha, label="Outages: All Causes (Outg Level)"))]
axs[2] = plot_multiple_annual_kwh_hist(axs[2], dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=f'All Outage Xfmrs - Density', fontsize=25), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                               dict(axis='y', labelsize=15)])
#----------------------------------------
#----------------------------------------
stat='count'
#--------------
dfs_with_args = [(dfs_outg_dict_by_mxo_prim['by_xfmr'],        dict(color='green', alpha=alpha, label="Outages: All Causes (Xfmr Level)")), 
                 (dfs_outg_dict_by_mxo_prim['by_outg'],        dict(color='darkgreen', alpha=alpha, label="Outages: All Causes (Outg Level)"))]
axs[1] = plot_multiple_annual_kwh_hist(axs[1], dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=f'Xfmr tied to Outg. Location ID - Count', fontsize=25), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                               dict(axis='y', labelsize=15)])


dfs_with_args = [(dfs_outg_dict_by_mxo['by_xfmr'],        dict(color='green', alpha=alpha, label="Outages: All Causes (Xfmr Level)")), 
                 (dfs_outg_dict_by_mxo['by_outg'],        dict(color='darkgreen', alpha=alpha, label="Outages: All Causes (Outg Level)"))]
axs[3] = plot_multiple_annual_kwh_hist(axs[3], dfs_w_args = dfs_with_args, 
                                    min_max_and_bin_size=min_max_and_bin_size, 
                                    include_over_underflow=include_over_underflow, stat=stat, plot_sns=False, 
                                    keep_edges_opaque=keep_edges_opaque, include_hatches=include_hatches,
                                    draw_side_by_side=draw_side_by_side, draw_single_idx_full_width=draw_single_idx_full_width,
                                    draw_legend=True, legend_args=dict(fontsize=15), 
                                    title_args=dict(label=f'All Outage Xfmrs - Count', fontsize=25), 
                                    ylabel_args = dict(ylabel=f'{stat.capitalize()}', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                    xlabel_args = dict(xlabel='Annual kWh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                    tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                               dict(axis='y', labelsize=15)])
#----------------------------------------
fig.suptitle('Annual kWh - Transformer vs. Outage Level', fontsize=40, 
             x=0.5*(subplots_adjust_args['left']+subplots_adjust_args['right']), ha='center')
#----------------------------------------
if save_figs:
    save_dir = os.path.join(save_dir_base_figs , 'annual_kwh', 'XfmrVsOutgLevel', 'PDF')
    save_name = 'annual_kwh_hist_all_causes.pdf'
    save_fig(fig, save_dir, save_name)
    #-----
    save_dir = os.path.join(save_dir_base_figs , 'annual_kwh', 'XfmrVsOutgLevel', 'PNG')
    save_name = 'annual_kwh_hist_all_causes.png'
    save_fig(fig, save_dir, save_name)
#----------------------------------------
fig_num+=1

In [ ]:
we1 = df_outage.groupby(['trsf_pole_nb', 'OUTG_REC_NB']).agg({'annual_kwh':'sum', 'annual_max_dmnd':'sum'})
we1

In [ ]:
we2 = df_outage.groupby(['OUTG_REC_NB', 'trsf_pole_nb']).agg({'annual_kwh':'sum', 'annual_max_dmnd':'sum'})
we2

In [ ]:
we2b = we2.groupby(level=0).agg({'annual_kwh':'mean', 'annual_max_dmnd':'mean'})
we2b

In [ ]:
we1.loc[('1615473439541', 12132222)]

In [ ]:
we2.loc[(12132222, '1615473439541')]

In [ ]:
cols_for_drop_duplicates = [x for x in df_outage.columns if x in cols_of_interest_met_prem]
xfmr_gpby_cols=['trsf_pole_nb']
xfmr_gpby_agg_dict={'annual_kwh':'sum', 'annual_max_dmnd':'sum'}
outg_gpby_cols=['OUTG_REC_NB']
outg_gpby_agg_dict={'annual_kwh':'sum', 'annual_max_dmnd':'sum'}

tmp_df_outage_by_meter = df_outage[cols_for_drop_duplicates].drop_duplicates().dropna(how='all')
tmp_df_outage_by_xfmr  = tmp_df_outage_by_meter.groupby(xfmr_gpby_cols).agg(xfmr_gpby_agg_dict)
tmp_df_outage_by_outg  = df_outage.groupby(outg_gpby_cols).agg(outg_gpby_agg_dict)

In [ ]:
tmp_df_outage_by_outg_v2  = df_outage.groupby(xfmr_gpby_cols).agg(outg_gpby_agg_dict)

In [ ]:
print(f'tmp_df_outage_by_meter.shape   = {tmp_df_outage_by_meter.shape}')
print(f'tmp_df_outage_by_xfmr.shape    = {tmp_df_outage_by_xfmr.shape}')
print(f'tmp_df_outage_by_outg.shape    = {tmp_df_outage_by_outg.shape}')
print(f'tmp_df_outage_by_outg_v2.shape = {tmp_df_outage_by_outg_v2.shape}')

In [ ]:
print(f"tmp_df_outage_by_meter['annual_kwh'].sum()   = {tmp_df_outage_by_meter['annual_kwh'].sum()}")
print(f"tmp_df_outage_by_xfmr['annual_kwh'].sum()    = {tmp_df_outage_by_xfmr['annual_kwh'].sum()}")
print(f"tmp_df_outage_by_outg['annual_kwh'].sum()    = {tmp_df_outage_by_outg['annual_kwh'].sum()}")
print(f"tmp_df_outage_by_outg_v2['annual_kwh'].sum() = {tmp_df_outage_by_outg_v2['annual_kwh'].sum()}")

In [ ]:
tmp_df_outage_by_outg[tmp_df_outage_by_outg['annual_kwh']<tmp_df_outage_by_outg['annual_kwh']]

In [ ]:
tmp_df_outage_by_outg[tmp_df_outage_by_outg['annual_kwh']<1e6]['annual_kwh'].hist(figsize=(14,6), color='red')

In [ ]:
tmp_df_outage_by_outg['annual_kwh'].describe()

In [ ]:
tmp_df_outage_by_xfmr['annual_kwh'].describe()

In [ ]:
bins_clip_minmax_dict = Plot_Hist.get_bins_clip_minmax((0.0, 2.5e5, 2.5e4/2.0), True, True, True)
tmp_df_outage_by_xfmr['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(figsize=(14,6), bins=bins_clip_minmax_dict['bins'], color='C0', label='By Xfmr', alpha=0.5)

In [ ]:
bins_clip_minmax_dict = Plot_Hist.get_bins_clip_minmax((0.0, 2.5e5, 2.5e4/2.0), True, True, True)
tmp_df_outage_by_outg['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(figsize=(14,6), bins=bins_clip_minmax_dict['bins'], color='red', label='By Outg', alpha=0.5)

In [ ]:
bins_clip_minmax_dict = Plot_Hist.get_bins_clip_minmax((0.0, 2.5e5, 2.5e4/2.0), True, True, True)
ax = tmp_df_outage_by_outg['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(figsize=(14,6), bins=bins_clip_minmax_dict['bins'], color='red', label='By Outg', alpha=0.5)
tmp_df_outage_by_xfmr['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(ax=ax, bins=bins_clip_minmax_dict['bins'], color='C0', label='By Xfmr', alpha=0.5)
ax.legend()

In [ ]:
bins_clip_minmax_dict = Plot_Hist.get_bins_clip_minmax((0.0, 2.5e5, 2.5e4/2.0), True, True, True)
ax = tmp_df_outage_by_outg['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(figsize=(14,6), bins=bins_clip_minmax_dict['bins'], color='red', label='By Outg', alpha=0.5)
tmp_df_outage_by_outg_v2['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(ax=ax, bins=bins_clip_minmax_dict['bins'], color='green', label='By Outg v2', alpha=0.5)
tmp_df_outage_by_xfmr['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(ax=ax, bins=bins_clip_minmax_dict['bins'], color='C0', label='By Xfmr', alpha=0.5)
ax.legend()

In [ ]:
bins_clip_minmax_dict = Plot_Hist.get_bins_clip_minmax((0.0, 2.5e6, 2.5e5/2.0), True, True, True)
ax = tmp_df_outage_by_outg['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(figsize=(14,6), bins=bins_clip_minmax_dict['bins'], color='red', label='By Outg', alpha=0.5)
tmp_df_outage_by_outg_v2['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(ax=ax, bins=bins_clip_minmax_dict['bins'], color='green', label='By Outg v2', alpha=0.5)
tmp_df_outage_by_xfmr['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(ax=ax, bins=bins_clip_minmax_dict['bins'], color='C0', label='By Xfmr', alpha=0.5)
ax.legend()

In [ ]:
bins_clip_minmax_dict = Plot_Hist.get_bins_clip_minmax((0.0, 2.5e5, 2.5e4/2.0), True, True, True)
ax = tmp_df_outage_by_outg['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(figsize=(14,6), bins=bins_clip_minmax_dict['bins'], color='red', label='By Outg', alpha=0.5)
tmp_df_outage_by_outg_v2['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(ax=ax, bins=bins_clip_minmax_dict['bins'], color='green', label='By Outg v2', alpha=0.5)
we2b['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(ax=ax, bins=bins_clip_minmax_dict['bins'], color='purple', label='we2b', alpha=0.5)
tmp_df_outage_by_xfmr['annual_kwh'].clip(bins_clip_minmax_dict['clip_min'], bins_clip_minmax_dict['clip_max']).hist(ax=ax, bins=bins_clip_minmax_dict['bins'], color='C0', label='By Xfmr', alpha=0.5)
ax.legend()

In [ ]:
we2b

# END DELETE

In [ ]:
print(dfs_outg_dict_by_mxo_prim['by_xfmr'][dfs_outg_dict_by_mxo_prim['by_xfmr']['annual_max_dmnd']==0].shape[0]/dfs_outg_dict_by_mxo_prim['by_xfmr'].shape[0])
print(dfs_outg_dict_by_mxo['by_xfmr'][dfs_outg_dict_by_mxo['by_xfmr']['annual_max_dmnd']==0].shape[0]/dfs_outg_dict_by_mxo['by_xfmr'].shape[0])

# EEMSP

### In EEMSP, LOCATION_NB can be used to join with trsf_pole_nb

In [ ]:
# import EEMSP
# reload(EEMSP)
# from EEMSP import EEMSP

In [ ]:
if not read_dfs_from_csv:
    df_construct_type=DFConstructType.kRunSqlQuery
    conn_eemsp = Utilities.get_eemsp_oracle_connection()
else:
    df_construct_type=DFConstructType.kReadCsv
    # Since I am being lazy and loading up contstruct_df_args with everything needed for csv or sql
    # conn_outages must always exist, even if it is simply None
    if 'conn_eemsp' not in locals():
        conn_eemsp = None
#-----
if save_dfs_to_csv:
    assert(df_construct_type!=DFConstructType.kReadCsv)

In [ ]:
#TODO REPLACE STUFF BELOW WITH SIMILAR TO:
# build_sql_function_kwargs = dict(trsf_pole_nbs=df_outage_save['trsf_pole_nb'].unique().tolist()[0:5000])
# eemsp = EEMSP(df_construct_type=DFConstructType.kRunSqlQuery, 
#               build_sql_function_kwargs=build_sql_function_kwargs, init_df_in_constructor=True)
# df_eemsp = eemsp.get_df()

In [ ]:
#--------------------------------------------------
# Build eemsp, eemsp_prim, eemsp_prim_strict, eemsp_no_outg 
#   and grab associated dataframes
#--------------------------------------------------
contstruct_df_args = dict(file_path=os.path.join(save_dir_csvs, 'df_eemsp_OG.csv'), 
                          conn_eemsp=conn_eemsp)
build_sql_function_kwargs = dict(trsf_pole_nbs=df_outage_save['trsf_pole_nb'].unique().tolist())
eemsp = EEMSP(
    df_construct_type=df_construct_type, 
    contstruct_df_args=contstruct_df_args, 
    build_sql_function_kwargs=build_sql_function_kwargs, 
    init_df_in_constructor=True
)
df_eemsp_OG = eemsp.get_df()
if save_dfs_to_csv:
    eemsp.write_df_to_csv(contstruct_df_args['file_path'])
#--------------------------------------------------
contstruct_df_args_prim = dict(file_path=os.path.join(save_dir_csvs, 'df_eemsp_prim_OG.csv'), 
                               conn_eemsp=conn_eemsp)
build_sql_function_kwargs_prim = dict(trsf_pole_nbs=df_outage_prim_save['trsf_pole_nb'].unique().tolist())
eemsp_prim = EEMSP(
    df_construct_type=df_construct_type, 
    contstruct_df_args=contstruct_df_args_prim, 
    build_sql_function_kwargs=build_sql_function_kwargs_prim, 
    init_df_in_constructor=True
)
df_eemsp_prim_OG = eemsp_prim.get_df()
if save_dfs_to_csv:
    df_eemsp_prim_OG.write_df_to_csv(contstruct_df_args_prim['file_path'])
#--------------------------------------------------
contstruct_df_args_prim_strict = dict(file_path=os.path.join(save_dir_csvs, 'df_eemsp_prim_strict_OG.csv'), 
                                      conn_eemsp=conn_eemsp)
build_sql_function_kwargs_prim_strict = dict(trsf_pole_nbs=df_outage_prim_strict_save['trsf_pole_nb'].unique().tolist())
eemsp_prim_strict = EEMSP(
    df_construct_type=df_construct_type, 
    contstruct_df_args=contstruct_df_args_prim_strict, 
    build_sql_function_kwargs=build_sql_function_kwargs_prim_strict, 
    init_df_in_constructor=True
)
df_eemsp_prim_strict_OG = eemsp_prim_strict.get_df()
if save_dfs_to_csv:
    df_eemsp_prim_strict_OG.write_df_to_csv(contstruct_df_args_prim_strict['file_path'])
#--------------------------------------------------
contstruct_df_args_no_outg = dict(file_path=os.path.join(save_dir_csvs, 'df_eemsp_no_outg_OG.csv'), 
                                  conn_eemsp=conn_eemsp)
build_sql_function_kwargs_no_outg = dict(trsf_pole_nbs=df_mp_no_outg_OG['trsf_pole_nb'].unique().tolist())
eemsp_no_outg = EEMSP(
    df_construct_type=df_construct_type, 
    contstruct_df_args=contstruct_df_args_no_outg, 
    build_sql_function_kwargs=build_sql_function_kwargs_no_outg, 
    init_df_in_constructor=True
)
df_eemsp_no_outg_OG = eemsp_no_outg.get_df()
if save_dfs_to_csv:
    df_eemsp_no_outg_OG.write_df_to_csv(contstruct_df_args_no_outg['file_path'])
#--------------------------------------------------

In [ ]:
df_eemsp = df_eemsp_OG.copy()
df_eemsp_prim = df_eemsp_prim_OG.copy()
df_eemsp_prim_strict = df_eemsp_prim_strict_OG.copy()
df_eemsp_no_outg = df_eemsp_no_outg_OG.copy()

In [ ]:
print(df_eemsp.shape)
print(df_eemsp.columns)

In [ ]:
#TODO Below statements made by looking at df_eemsp
#     Expect df_eemsp_no_outg to follow same, but haven't checked
#TODO NOTE: There are a few less SERIAL_NB than entries!
# Same number entries as: LOCATION_NB, LOCATION_ID, 'EQSEQ_ID', 'EQUIP_ID'
#                         TRANSACTION_ID, and possibly/probably more
#
#
# TODO should I check e.g. CIRCUIT_NB/GIS_CIRCUIT_NB etc. when merging?
#   and/or use e.g. CIRCUIT_NB/GIS_CIRCUIT_NB etc. in the merge?

# NOTE: 'INSTALL_DT_YEAR' was built by me, not in std table
cols_of_interest_eemsp = [
    'LOCATION_NB', 'INSTALL_DT', 'INSTALL_DT_YEAR', 'REMOVAL_DT', 
    'LATEST_STATUS', 'MFGR_NM', 'LAST_TRANS_DESC','COOLANT',
    'INFO', 'KVA_SIZE', 'PHASE_CNT', 'PRIM_VOLTAGE', 'SEC_VOLTAGE', 'PROTECTION',
    'PRU_NUMBER', 'TAPS', 'XFTYPE',
]

In [ ]:
df_eemsp['INSTALL_DT_YEAR'] = df_eemsp["INSTALL_DT"].dt.year
df_eemsp_prim['INSTALL_DT_YEAR'] = df_eemsp_prim["INSTALL_DT"].dt.year
df_eemsp_prim_strict['INSTALL_DT_YEAR'] = df_eemsp_prim_strict["INSTALL_DT"].dt.year
df_eemsp_no_outg['INSTALL_DT_YEAR'] = df_eemsp_no_outg["INSTALL_DT"].dt.year

In [ ]:
df_eemsp = df_eemsp[cols_of_interest_eemsp]
df_eemsp_prim = df_eemsp_prim[cols_of_interest_eemsp]
df_eemsp_prim_strict = df_eemsp_prim_strict[cols_of_interest_eemsp]
df_eemsp_no_outg = df_eemsp_no_outg[cols_of_interest_eemsp]

In [ ]:
# df_outage_by_xfmr_cp = dfs_outg_dict_by_mxo['by_xfmr'].copy()
# df_outage_by_xfmr_cp = df_outage_by_xfmr_cp.reset_index()
# df_outage_by_xfmr_cp = df_outage_by_xfmr_cp.merge(df_eemsp, how='left', 
#                                                   left_on='trsf_pole_nb', right_on='LOCATION_NB', indicator=True)
# label_append = 'All Xfmrs'

df_outage_by_xfmr_cp = dfs_outg_dict_by_mxo_prim_strict['by_xfmr'].copy()
df_outage_by_xfmr_cp = df_outage_by_xfmr_cp.reset_index()
df_outage_by_xfmr_cp = df_outage_by_xfmr_cp.merge(df_eemsp_prim_strict, how='left', 
                                                  left_on='trsf_pole_nb', right_on='LOCATION_NB', indicator=True)
label_append = 'Direct Xfmrs'

In [ ]:
df_mp_no_outg_by_xfmr_cp = dfs_mp_no_outg_dict_by_mx['by_xfmr'].copy()
df_mp_no_outg_by_xfmr_cp = df_mp_no_outg_by_xfmr_cp.reset_index()
df_mp_no_outg_by_xfmr_cp = df_mp_no_outg_by_xfmr_cp.merge(df_eemsp_no_outg, how='left', 
                                                          left_on='trsf_pole_nb', right_on='LOCATION_NB', indicator=True)

In [ ]:
df_mp_no_outg_by_xfmr_cp

In [ ]:
df_mp_no_outg_by_xfmr_cp['_merge'].value_counts()

In [ ]:
print(dfs_mp_no_outg_dict_by_mx['by_xfmr'].index.nunique())
print(df_eemsp_no_outg['LOCATION_NB'].nunique())
print(df_mp_no_outg_by_xfmr_cp[df_mp_no_outg_by_xfmr_cp['_merge']=='left_only'].shape[0])

In [ ]:
df_outage_by_xfmr_cp['_merge'].value_counts()

In [ ]:
print(dfs_outg_dict_by_mxo['by_xfmr'].index.nunique())
print(df_eemsp['LOCATION_NB'].nunique())
print(df_outage_by_xfmr_cp[df_outage_by_xfmr_cp['_merge']=='left_only'].shape[0])

In [ ]:
df_outage_by_xfmr_cp = df_outage_by_xfmr_cp[df_outage_by_xfmr_cp['_merge']=='both'].copy()
df_mp_no_outg_by_xfmr_cp = df_mp_no_outg_by_xfmr_cp[df_mp_no_outg_by_xfmr_cp['_merge']=='both'].copy()

# MFGR_NM

In [ ]:
mfgr_nm_val_counts_outage = df_outage_by_xfmr_cp['MFGR_NM'].value_counts()
mfgr_nm_val_counts_no_outg = df_mp_no_outg_by_xfmr_cp['MFGR_NM'].value_counts()
#----
mfgr_nm_val_counts_pct_outage = mfgr_nm_val_counts_outage/df_outage_by_xfmr_cp.shape[0]
mfgr_nm_val_counts_pct_no_outg = mfgr_nm_val_counts_no_outg/df_mp_no_outg_by_xfmr_cp.shape[0]

In [ ]:
shared_mfgrs = [x for x in mfgr_nm_val_counts_no_outg.index 
                if x in mfgr_nm_val_counts_outage.index]
#-----
ratio_outage_to_no_outg_mfgr_nm = mfgr_nm_val_counts_outage[shared_mfgrs]/mfgr_nm_val_counts_no_outg[shared_mfgrs]
ratio_outage_to_no_outg_mfgr_nm = ratio_outage_to_no_outg_mfgr_nm.sort_values(ascending=False)
#-----
ratio_pct_outage_to_no_outg_mfgr_nm = mfgr_nm_val_counts_pct_outage[shared_mfgrs]/mfgr_nm_val_counts_pct_no_outg[shared_mfgrs]
ratio_pct_outage_to_no_outg_mfgr_nm = ratio_pct_outage_to_no_outg_mfgr_nm.sort_values(ascending=False)

# PRIM_VOLTAGE
NOTE: Cannot simply call .hist for these, since binning by category not variable.
Even if you make the number of bins equal to the number of categories, it gives somewhat strange results.
One could maybe enumerate the categories and then hist?  But the method below is simpler and cleaner

In [ ]:
prim_vltg_val_counts_outage = df_outage_by_xfmr_cp['PRIM_VOLTAGE'].value_counts()
prim_vltg_val_counts_no_outg = df_mp_no_outg_by_xfmr_cp['PRIM_VOLTAGE'].value_counts()
#----
prim_vltg_val_counts_pct_outage = prim_vltg_val_counts_outage/df_outage_by_xfmr_cp.shape[0]
prim_vltg_val_counts_pct_no_outg = prim_vltg_val_counts_no_outg/df_mp_no_outg_by_xfmr_cp.shape[0]

In [ ]:
shared_mfgrs = [x for x in prim_vltg_val_counts_no_outg.index 
                if x in prim_vltg_val_counts_outage.index]
#-----
ratio_outage_to_no_outg_prim_vltg = prim_vltg_val_counts_outage[shared_mfgrs]/prim_vltg_val_counts_no_outg[shared_mfgrs]
ratio_outage_to_no_outg_prim_vltg = ratio_outage_to_no_outg_prim_vltg.sort_values(ascending=False)
#-----
ratio_pct_outage_to_no_outg_prim_vltg = prim_vltg_val_counts_pct_outage[shared_mfgrs]/prim_vltg_val_counts_pct_no_outg[shared_mfgrs]
ratio_pct_outage_to_no_outg_prim_vltg = ratio_pct_outage_to_no_outg_prim_vltg.sort_values(ascending=False)

# SEC_VOLTAGE

In [ ]:
sec_vltg_val_counts_outage = df_outage_by_xfmr_cp['SEC_VOLTAGE'].value_counts()
sec_vltg_val_counts_no_outg = df_mp_no_outg_by_xfmr_cp['SEC_VOLTAGE'].value_counts()
#----
sec_vltg_val_counts_pct_outage = sec_vltg_val_counts_outage/df_outage_by_xfmr_cp.shape[0]
sec_vltg_val_counts_pct_no_outg = sec_vltg_val_counts_no_outg/df_mp_no_outg_by_xfmr_cp.shape[0]

In [ ]:
shared_mfgrs = [x for x in sec_vltg_val_counts_no_outg.index 
                if x in sec_vltg_val_counts_outage.index]
#-----
ratio_outage_to_no_outg_sec_vltg = sec_vltg_val_counts_outage[shared_mfgrs]/sec_vltg_val_counts_no_outg[shared_mfgrs]
ratio_outage_to_no_outg_sec_vltg = ratio_outage_to_no_outg_sec_vltg.sort_values(ascending=False)
#-----
ratio_pct_outage_to_no_outg_sec_vltg = sec_vltg_val_counts_pct_outage[shared_mfgrs]/sec_vltg_val_counts_pct_no_outg[shared_mfgrs]
ratio_pct_outage_to_no_outg_sec_vltg = ratio_pct_outage_to_no_outg_sec_vltg.sort_values(ascending=False)

# KVA_SIZE

In [ ]:
df_outage_by_xfmr_cp['KVA_SIZE'] = df_outage_by_xfmr_cp['KVA_SIZE'].astype(np.float64)
df_mp_no_outg_by_xfmr_cp['KVA_SIZE'] = df_mp_no_outg_by_xfmr_cp['KVA_SIZE'].astype(np.float64)

# Join outage and eemsp

In [ ]:
# Might make sense to join df_eemsp with df_outage without PREMISE_NB information
# i.e. after just a single row per outage in df_outage
# Could also use a consolidate type routine to build such a df_outage to use
df_outage_w_eemsp = df_outage.copy()
df_outage_w_eemsp = df_outage_w_eemsp.merge(df_eemsp, how='left', 
                                            left_on='trsf_pole_nb', right_on='LOCATION_NB', 
                                            indicator=True)
df_outage_w_eemsp['DT_OFF_TS_FULL'] = pd.to_datetime(df_outage_w_eemsp['DT_OFF_TS_FULL'])
df_outage_w_eemsp['age_at_outage'] = df_outage_w_eemsp['DT_OFF_TS_FULL']-df_outage_w_eemsp['INSTALL_DT']
df_outage_w_eemsp['age_at_outage_days'] = df_outage_w_eemsp['age_at_outage'].dt.round('D').dt.days
df_outage_w_eemsp['age_at_outage_years'] = df_outage_w_eemsp['age_at_outage_days'].div(365.25)

In [ ]:
df_outage_w_eemsp_prim = df_outage_prim.copy()
df_outage_w_eemsp_prim = df_outage_w_eemsp_prim.merge(df_eemsp_prim, how='left', 
                                            left_on='trsf_pole_nb', right_on='LOCATION_NB', 
                                            indicator=True)
df_outage_w_eemsp_prim['DT_OFF_TS_FULL'] = pd.to_datetime(df_outage_w_eemsp_prim['DT_OFF_TS_FULL'])
df_outage_w_eemsp_prim['age_at_outage'] = df_outage_w_eemsp_prim['DT_OFF_TS_FULL']-df_outage_w_eemsp_prim['INSTALL_DT']
df_outage_w_eemsp_prim['age_at_outage_days'] = df_outage_w_eemsp_prim['age_at_outage'].dt.round('D').dt.days
df_outage_w_eemsp_prim['age_at_outage_years'] = df_outage_w_eemsp_prim['age_at_outage_days'].div(365.25)

In [ ]:
df_outage_w_eemsp_prim_strict = df_outage_prim_strict.copy()
df_outage_w_eemsp_prim_strict = df_outage_w_eemsp_prim_strict.merge(df_eemsp_prim_strict, how='left', 
                                            left_on='trsf_pole_nb', right_on='LOCATION_NB', 
                                            indicator=True)
df_outage_w_eemsp_prim_strict['DT_OFF_TS_FULL'] = pd.to_datetime(df_outage_w_eemsp_prim_strict['DT_OFF_TS_FULL'])
df_outage_w_eemsp_prim_strict['age_at_outage'] = df_outage_w_eemsp_prim_strict['DT_OFF_TS_FULL']-df_outage_w_eemsp_prim_strict['INSTALL_DT']
df_outage_w_eemsp_prim_strict['age_at_outage_days'] = df_outage_w_eemsp_prim_strict['age_at_outage'].dt.round('D').dt.days
df_outage_w_eemsp_prim_strict['age_at_outage_years'] = df_outage_w_eemsp_prim_strict['age_at_outage_days'].div(365.25)

In [ ]:
df_outage_w_eemsp_prim_strict.head()

In [ ]:
df_outage_w_eemsp_prim = df_outage_w_eemsp[df_outage_w_eemsp['LOCATION_ID']==df_outage_w_eemsp['trsf_pole_nb']].copy()

df_outage_w_eemsp_by_meter = df_outage_w_eemsp.drop_duplicates(subset=[x for x in df_outage_w_eemsp.columns if x in cols_of_interest_met_prem]).dropna(how='all')
df_outage_w_eemsp_by_meter_prim = df_outage_w_eemsp_prim.drop_duplicates(subset=[x for x in df_outage_w_eemsp_prim.columns if x in cols_of_interest_met_prem]).dropna(how='all')
#-----
xfmr_equip_typ_nms_of_interest = ['TRANSFORMER, OH', 'TRANSFORMER, UG']
df_outage_w_eemsp_by_meter_prim_strict = df_outage_w_eemsp_by_meter_prim[df_outage_w_eemsp_by_meter_prim['EQUIP_TYP_NM'].isin(xfmr_equip_typ_nms_of_interest)]
#-----
df_outage_w_eemsp_by_xfmr = df_outage_w_eemsp_by_meter.groupby('trsf_pole_nb')[['annual_kwh', 'annual_max_dmnd']].sum()
df_outage_w_eemsp_by_xfmr_prim = df_outage_w_eemsp_by_meter_prim.groupby('trsf_pole_nb')[['annual_kwh', 'annual_max_dmnd']].sum()
df_outage_w_eemsp_by_xfmr_prim_strict = df_outage_w_eemsp_by_meter_prim_strict.groupby('trsf_pole_nb')[['annual_kwh', 'annual_max_dmnd']].sum()

In [ ]:
df_outage_w_eemsp_dl_ol = df_outage_w_eemsp[(df_outage_w_eemsp['MJR_CAUSE_CD']=='DL') & 
                                            (df_outage_w_eemsp['MNR_CAUSE_CD']=='OL')].copy()
df_outage_w_eemsp_dl_ol_prim =  df_outage_w_eemsp_dl_ol[df_outage_w_eemsp_dl_ol['LOCATION_ID']==df_outage_w_eemsp_dl_ol['trsf_pole_nb']].copy()

df_outage_w_eemsp_dl_ol_by_meter = df_outage_w_eemsp_dl_ol.drop_duplicates(subset=[x for x in df_outage_w_eemsp.columns if x in cols_of_interest_met_prem]).dropna(how='all')
df_outage_w_eemsp_dl_ol_by_meter_prim = df_outage_w_eemsp_dl_ol_prim.drop_duplicates(subset=[x for x in df_outage_w_eemsp.columns if x in cols_of_interest_met_prem]).dropna(how='all')
df_outage_w_eemsp_dl_ol_by_meter_prim_strict = df_outage_w_eemsp_dl_ol_by_meter_prim[df_outage_w_eemsp_dl_ol_by_meter_prim['EQUIP_TYP_NM'].isin(xfmr_equip_typ_nms_of_interest)]
#-----
df_outage_w_eemsp_dl_ol_by_xfmr = df_outage_w_eemsp_dl_ol_by_meter.groupby('trsf_pole_nb')[['annual_kwh', 'annual_max_dmnd']].sum()
df_outage_w_eemsp_dl_ol_by_xfmr_prim = df_outage_w_eemsp_dl_ol_by_meter_prim.groupby('trsf_pole_nb')[['annual_kwh', 'annual_max_dmnd']].sum()
df_outage_w_eemsp_dl_ol_by_xfmr_prim_strict = df_outage_w_eemsp_dl_ol_by_meter_prim_strict.groupby('trsf_pole_nb')[['annual_kwh', 'annual_max_dmnd']].sum()

In [ ]:
df_outage_w_eemsp_dl_eqf = df_outage_w_eemsp[(df_outage_w_eemsp['MJR_CAUSE_CD']=='DL') & 
                                            (df_outage_w_eemsp['MNR_CAUSE_CD']=='EQF')].copy()
df_outage_w_eemsp_dl_eqf_prim =  df_outage_w_eemsp_dl_eqf[df_outage_w_eemsp_dl_eqf['LOCATION_ID']==df_outage_w_eemsp_dl_eqf['trsf_pole_nb']].copy()

df_outage_w_eemsp_dl_eqf_by_meter = df_outage_w_eemsp_dl_eqf.drop_duplicates(subset=[x for x in df_outage_w_eemsp.columns if x in cols_of_interest_met_prem]).dropna(how='all')
df_outage_w_eemsp_dl_eqf_by_meter_prim = df_outage_w_eemsp_dl_eqf_prim.drop_duplicates(subset=[x for x in df_outage_w_eemsp.columns if x in cols_of_interest_met_prem]).dropna(how='all')
df_outage_w_eemsp_dl_eqf_by_meter_prim_strict = df_outage_w_eemsp_dl_eqf_by_meter_prim[df_outage_w_eemsp_dl_eqf_by_meter_prim['EQUIP_TYP_NM'].isin(xfmr_equip_typ_nms_of_interest)]
#-----
df_outage_w_eemsp_dl_eqf_by_xfmr = df_outage_w_eemsp_dl_eqf_by_meter.groupby('trsf_pole_nb')[['annual_kwh', 'annual_max_dmnd']].sum()
df_outage_w_eemsp_dl_eqf_by_xfmr_prim = df_outage_w_eemsp_dl_eqf_by_meter_prim.groupby('trsf_pole_nb')[['annual_kwh', 'annual_max_dmnd']].sum()
df_outage_w_eemsp_dl_eqf_by_xfmr_prim_strict = df_outage_w_eemsp_dl_eqf_by_meter_prim_strict.groupby('trsf_pole_nb')[['annual_kwh', 'annual_max_dmnd']].sum()

# Misc.

In [ ]:
# This basically shows the number of outages suffered over the time period collected
# The number of outages suffered over some past amount of time may be a good feature for model
# NOTE: result should be same whether grouping by LOCATION_ID or trsf_pole_nb
n_outages_per_xfmr = df_outage_prim.groupby('LOCATION_ID')['OUTG_REC_NB'].nunique().to_dict()

In [ ]:
n_outages_per_xfmr

In [ ]:
pole_prems_dict = {}
for idx, gp in df_mp_all.groupby('trsf_pole_nb'):
    assert(idx not in pole_prems_dict)
    pole_prems_dict[idx] = gp['prem_nb'].tolist()

In [ ]:
pole_prems_dict_rev = {}
for trsf_pole_nb, list_of_prems in pole_prems_dict.items():
    for prem in list_of_prems:
        if prem in pole_prems_dict_rev:
            assert(pole_prems_dict_rev[prem]==trsf_pole_nb)
            continue
        #assert(prem not in pole_prems_dict_rev)
        pole_prems_dict_rev[prem] = trsf_pole_nb

In [ ]:
pole_prems_dict

In [ ]:
pole_prems_dict_rev

# 2D heatmap

In [ ]:
def plot_2d_heatmap(ax, df, x_col, y_col, 
                    min_max_and_bin_size_x, min_max_and_bin_size_y, 
                    include_over_underflow=False, stat='count', 
                    discrete_x=False, discrete_y=False, 
                    draw_legend=False, 
                    hist_plot_kwargs=None, 
                    **kwargs):
    # If min_max_and_bin_size is None, use default number of bins in pandas.DataFrame.hist, which is 10
    # If min_max_and_bin_size is an integer, take that the be the number of bins
    # Otherwise, min_max_and_bin_size must be a list/tuple of length 3 containing 
    #   min_in_plot, max_in_plot, and bin_size
    #
    # include_over_underflow:
    #   This can be a single boolean, or a list/tuple of two booleans
    #   If single boolean: include_overflow = include_underflow = include_over_underflow
    #   If pair of booleans: include_overflow = include_over_underflow[0], 
    #                        include_underflow = include_over_underflow[1]
    #
    #
    # Possible values for stat:
    #     count: show the number of observations in each bin
    #     frequency: show the number of observations divided by the bin width
    #     probability: or proportion: normalize such that bar heights sum to 1
    #     density: normalize such that the total area of the histogram equals 1
    #---------------------------
    assert(stat in ['count', 'frequency', 'probability', 'density'])
    #---------------------------
    assert(isinstance(include_over_underflow, bool) or 
           isinstance(include_over_underflow, list) or 
           isinstance(include_over_underflow, tuple))
    if isinstance(include_over_underflow, bool):
        include_overflow = include_underflow = include_over_underflow
    else:
        include_overflow = include_over_underflow[0]
        include_underflow = include_over_underflow[1]
    #---------------------------
    title_args  = kwargs.get('title_args', None)
    ax_args     = kwargs.get('ax_args', None)
    xlabel_args = kwargs.get('xlabel_args', None)
    ylabel_args = kwargs.get('ylabel_args', None)
    legend_args = kwargs.get('legend_args', None)
    tick_args = kwargs.get('tick_args', None)
    # Note: tick_args can be a dict or a list
    #       Making it a list allows operations on both x and y
    #       e.g. tick_args =[dict(axis='x', labelrotation=90, labelsize=7.0, direction='in'), 
    #                        dict(axis='y', labelrotation=0, labelsize=10.0, direction='out')]
    allow_max_to_expand_to_fit_int_n_bins = kwargs.get('allow_max_to_expand_to_fit_int_n_bins', True)
    #---------------------------
    if hist_plot_kwargs is None:
        hist_plot_kwargs = {}
    #---------------------------
    clip_min_x = -np.inf
    clip_max_x = np.inf    
    if min_max_and_bin_size_x is None:
        bins_x=10 # default value
    elif isinstance(min_max_and_bin_size_x, int):
        bins_x = min_max_and_bin_size_x
    else:
        bins_clip_minmax_dict_x = Plot_Hist.get_bins_clip_minmax(min_max_and_bin_size_x, 
                                                                 include_overflow, include_underflow, 
                                                                 allow_max_to_expand_to_fit_int_n_bins)
        bins_x     = bins_clip_minmax_dict_x['bins']
        clip_min_x = bins_clip_minmax_dict_x['clip_min']
        clip_max_x = bins_clip_minmax_dict_x['clip_max']
    #---------------------------
    clip_min_y = -np.inf
    clip_max_y = np.inf    
    if min_max_and_bin_size_y is None:
        bins_y=10 # default value
    elif isinstance(min_max_and_bin_size_y, int):
        bins_y = min_max_and_bin_size_y
    else:
        bins_clip_minmax_dict_y = Plot_Hist.get_bins_clip_minmax(min_max_and_bin_size_y, 
                                                                 include_overflow, include_underflow, 
                                                                 allow_max_to_expand_to_fit_int_n_bins)
        bins_y     = bins_clip_minmax_dict_y['bins']
        clip_min_y = bins_clip_minmax_dict_y['clip_min']
        clip_max_y = bins_clip_minmax_dict_y['clip_max']          
    
    #---------------------------    
    # TODO DO I need to copy, or no?
    df_to_plot = df[[x_col, y_col]].copy()
    df_to_plot[x_col] = df_to_plot[x_col].clip(clip_min_x, clip_max_x)
    df_to_plot[y_col] = df_to_plot[y_col].clip(clip_min_y, clip_max_y)
    #-----
    sns.histplot(ax=ax, data=df_to_plot, x=x_col, y=y_col,
                 bins=(bins_x, bins_y), stat=stat, discrete=(discrete_x, discrete_y), 
                 **hist_plot_kwargs)
    #----------------------------
    if isinstance(title_args, str):
        title_args = dict(label=title_args)
    if title_args is not None:
        ax.set_title(**title_args)
    #----------------------------
    if ax_args is not None:
        ax.set(**ax_args)
    if xlabel_args is not None:
        ax.set_xlabel(**xlabel_args)
    if ylabel_args is not None:
        ax.set_ylabel(**ylabel_args)
    if tick_args is not None:
        if isinstance(tick_args, dict):
            tick_args = [tick_args]
        for t_args in tick_args:
            ax.tick_params(**t_args)
    #---------------------------
    if draw_legend:
        if legend_args is None:
            ax.legend()
        else:
            ax.legend(**legend_args)
    #---------------------------
    # Draw line to distinguish over- and underflow
    if((isinstance(min_max_and_bin_size, list) or isinstance(min_max_and_bin_size, tuple)) and
      (include_overflow or include_underflow)):
        if include_underflow:
            ax.axvline(bins_x[1], color='black', linestyle='dashdot')
            ax.axhline(bins_y[1], color='black', linestyle='dashdot')
        if include_overflow:
            ax.axvline(bins_x[-2], color='black', linestyle='dashdot')
            ax.axhline(bins_y[-2], color='black', linestyle='dashdot')
    ax.grid(False)
    #---------------------------
    return ax

In [ ]:
tmp_df_outage_prim_strict = df_outage_prim_strict.copy()
print(tmp_df_outage_prim_strict.shape)
tmp_df_outage_prim_strict

In [ ]:
tmp_df_outage_prim_strict = tmp_df_outage_prim_strict.drop_duplicates(subset=['OUTG_REC_NB'])
print(tmp_df_outage_prim_strict.shape)

In [ ]:
tmp_df_outage_prim_strict['trsf_pole_nb'].nunique()

In [ ]:
tmp_df_outage_prim_strict['trsf_pole_nb'].value_counts()>1

In [ ]:
tmp_srs = tmp_df_outage_prim_strict['trsf_pole_nb'].value_counts()
multiple_outages = tmp_srs[tmp_srs>1].index

In [ ]:
multiple_outages

In [ ]:
tmp_df_outage_prim_strict[tmp_df_outage_prim_strict['trsf_pole_nb'].isin(multiple_outages)]['annual_kwh'].describe()

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.histplot(ax=ax0, data=tmp_df_outage_prim_strict['trsf_pole_nb'].value_counts(), discrete=True)
ax0.set_title('Number of outages per transformer')
ax0.set_yscale('log')

In [ ]:
tmp_agg = tmp_df_outage_prim_strict.groupby('trsf_pole_nb').agg({'annual_kwh':['sum', 'count'], 
                                                       'annual_max_dmnd':['sum', 'count'], 
                                                       'CI_NB':['sum', 'count'], 
                                                       'CMI_NB':['sum', 'count']})

In [ ]:
min_max_and_bin_size_x = 30
min_max_and_bin_size_y = None 
include_overflow=False
include_underflow=False
allow_max_to_expand_to_fit_int_n_bins=True
x_col = ('annual_kwh', 'sum')
y_col = ('annual_kwh', 'count')
stat = 'count'

fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
ax0 = plot_2d_heatmap(ax=ax0, df=tmp_agg, 
                      x_col=x_col, y_col=y_col, 
                      min_max_and_bin_size_x=min_max_and_bin_size_x, min_max_and_bin_size_y=min_max_and_bin_size_y, 
                      include_over_underflow=[include_overflow, include_underflow], stat=stat, 
                      hist_plot_kwargs = dict(cbar=True, cbar_kws=dict(shrink=.75)), 
                      title_args=dict(label=f'n_outages vs annual_kwh', fontsize=20), 
                      ylabel_args = dict(ylabel='n_outages', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                      xlabel_args = dict(xlabel='annual_kwh', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                      tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                 dict(axis='y', labelsize=15)],                       
                      discrete_y=True)

In [ ]:
min_max_and_bin_size_x = (1, 3, 1)
min_max_and_bin_size_y = (0, 1000, 200)
include_overflow=True
include_underflow=False
allow_max_to_expand_to_fit_int_n_bins=True
x_col = ('CI_NB', 'sum')
y_col = ('CMI_NB', 'sum')
stat = 'count'

fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
ax0 = plot_2d_heatmap(ax=ax0, df=tmp_agg, x_col=x_col, y_col=y_col, 
                      min_max_and_bin_size_x=min_max_and_bin_size_x, min_max_and_bin_size_y=min_max_and_bin_size_y, 
                      include_over_underflow=[include_overflow, include_underflow], stat=stat, 
                      hist_plot_kwargs = dict(cbar=True, cbar_kws=dict(shrink=.75)), 
                      title_args=dict(label=f'CMI_NB vs CI_NB', fontsize=20), 
                      ylabel_args = dict(ylabel='CMI_NB', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                      xlabel_args = dict(xlabel='CI_NB', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                      tick_args=[dict(axis='x', labelrotation=0, labelsize=15), 
                                 dict(axis='y', labelsize=15)])